In [47]:
import pandas as pd
import numpy as np
import datetime as dt
from scipy.stats import norm
from pandas.tseries.offsets import *
from scipy import optimize
from tqdm import tqdm


def calcVolatility(x): 
    return np.std(np.log(np.divide(x[1:],x[:-1])))*np.sqrt(252)

def BlackScholesCallValue(S,X,r,sigma,T):
    d1=(np.log(S/X)+(r+0.5*sigma**2)*T)/(sigma*np.sqrt(T));
    d2=d1-sigma*np.sqrt(T);
    delta=norm.cdf(d1)
    CP=S*delta-X*np.exp(-r*T)*norm.cdf(d2);
    return [CP, delta]  

def BlackScholesZero(C,X,r,sigma,T):
    # solves Black Scholes Call option formula for S using Newton's method
    if X==0:
        return C

    UpperS=C+X*np.exp(-r*T);
    if sigma==0 or X==0:
        return UpperS

    LowerS=C
    tol=0.0001*C
    delta=C; itercount=0; x0=LowerS; x1=UpperS
    while abs(delta)>tol and itercount<=100:
        itercount=itercount+1
        [bs0,D0]=BlackScholesCallValue(x0, X, r, sigma, T)
        y0=bs0-C
        [bs1,D1]=BlackScholesCallValue(x1, X, r, sigma, T)
        y1=bs1-C

        if D0<0.01:
            xguess=(x0*y1-x1*y0)/(y1-y0);
        else:
            xguess = x0 - y0/D0;

        if xguess>UpperS:
            xguess=UpperS
        elif xguess<LowerS:
            xguess=LowerS
        [bsxg,Dxg]=BlackScholesCallValue(xguess,X,r,sigma,T)
        delta=bsxg-C
        x0=xguess

            
    if abs(delta)>tol:
        return np.nan
    else:
        return xguess

def dailyDLIcalcs(Ve,X,r,sigma_a,T):
    Va=[]
    for v in Ve:
        Va.append(BlackScholesZero(v,X,r,sigma_a,T))
    Va=np.array(Va)
    return [Va, calcVolatility(Va)]

def calc_DD(rf, X, Ve, T):
    sigma_a=calcVolatility(Ve)
    sigma_previous=sigma_a

    delta=1.0
    itercount=0

    while not np.isnan(delta) and delta>0.0001 and itercount<100:
        itercount=itercount+1
        [Va, sigma_a]=dailyDLIcalcs(Ve, X, rf, sigma_a, T)
        delta=abs(sigma_a-sigma_previous)
        sigma_previous=sigma_a

    # compute drift term
    mu=np.mean(np.log(np.divide(Va[1:],Va[:-1])))

    if X==0: # no probability of default
        DD=100
    elif sigma_a==0: # if stock is not traded
        DD=np.nan
    else:
        DD=(np.log(Va[-1]/X)+(mu-(0.5*sigma_a**2))*T) / (sigma_a*np.sqrt(T))

    p_def = norm.cdf(-DD)

    return [p_def, DD, itercount, delta]

In [56]:

Ve = [ 6022.5, 6022.5, 7391.25, 7938.75, 9307.5   ,  12045.,
 11497.5   ,  11223.75    ,12045.     , 11497.5    , 11497.5   ,  11223.75,
 10676.25   , 10402.5     , 9307.5    ,  9718.125  ,  9307.5   ,   9033.75,
  8760.       ,8760.      , 9033.75   ,  9033.75   ,  8760.    ,   6843.75,
  7665.       ,7938.75    , 7938.75   ,  7938.75   ,  7117.5   ,   7117.5,
  7117.5      ,7117.5     , 8212.5   ,   9855.   ,   10128.75    ,10060.3125,
 10128.75     ,9581.25     ,9444.375    ,8896.875   , 8486.25    , 8623.125,
  8486.25     ,8349.375    ,8486.25     ,8486.25    , 8486.25    , 8486.25,
  8212.5      ,7938.75     ,7938.75     ,7938.75    , 7665.      , 7254.375,
  6843.75     ,6980.625    ,6980.625    ,7254.375   , 6980.625   , 7254.375,
  8212.5      ,8623.125    ,8212.5      ,8075.625   , 8212.5     , 8349.375,
  8212.5      ,7391.25     ,7254.375    ,7254.375   , 7528.125   , 7391.25,
  7391.25     ,7391.25     ,7391.25     ,7391.25    , 7391.25    , 7391.25,
  7391.25     ,7391.25     ,7391.25     ,7391.25    , 7391.25    , 7117.5,
  6980.625    ,6159.375    ,6159.375    ,5748.75    , 6022.5     , 6022.5,
  5489.21875  ,5489.21875  ,5489.21875  ,5744.53125 , 5872.1875  , 5999.84375,
  5872.1875   ,5744.53125  ,5744.53125  ,5744.53125 , 5744.53125 , 5744.53125,
  5744.53125  ,5744.53125  ,5361.5625   ,5361.5625  , 5233.90625 , 5106.25,
  4467.96875  ,5106.25     ,5106.25     ,5106.25    , 4978.59375 , 4978.59375,
  5106.25     ,5106.25     ,4978.59375  ,4850.9375  , 4340.3125  , 4340.3125,
  4467.96875  ,4340.3125   ,4340.3125   ,4340.3125  , 4340.3125  , 4340.3125,
  4340.3125   ,4212.65625  ,4723.28125  ,4850.9375  , 4978.59375 , 4723.28125,
  4723.28125  ,4723.28125  ,4723.28125  ,4723.28125 , 4723.28125 , 4850.9375,
  4850.9375   ,4850.9375   ,4850.9375   ,4850.9375  , 4850.9375  , 4850.9375,
  4850.9375   ,4850.9375   ,4085.       ,4085.      , 4085.      , 3829.6875,
  3829.6875   ,3829.6875   ,3829.6875   ,3834.375   , 3834.375   , 3323.125,
  3706.5625   ,3706.5625   ,3834.375    ,3834.375   , 3834.375   , 3962.1875,
  3834.375    ,3834.375    ,3706.5625   ,3962.1875  , 4090.      , 4856.875,
  5112.5      ,5240.3125   ,5368.125    ,5368.125   , 4984.6875  , 4856.875,
  5751.5625   ,6774.0625   ,7796.5625   ,8819.0625  ,10991.875   ,11886.5625,
 12014.375  , 11630.9375 , 13548.125   ,15081.875  , 14826.25  ,  16232.1875,
 16487.8125 , 17893.75   , 19555.3125  ,19171.875  , 18277.1875,  19044.0625,
 18788.4375 , 18788.4375 , 18788.4375  ,19044.0625 , 19299.6875,  20577.8125,
 20577.8125 , 21344.6875 , 22239.375   ,23900.9375 , 26201.5625,  25945.9375,
 26329.375  , 26329.375  , 26329.375   ,26329.375  , 26201.5625,  26840.625,
 27607.5    , 29013.4375 , 29780.3125  ,29652.5    , 29141.25  ,  27863.125,
 28196.25   , 28711.25   , 29226.25    ,30256.25   , 30256.25  ,  29998.75,
 29870.     , 28968.75   , 28582.5     ,28711.25   , 28582.5   ,  28582.5,
 27552.5    , 28067.5    , 29612.5     ,28067.5    , 24333.75  ,  25235.,
 25106.25   , 23947.5    , 21243.75    ,    0     , 14806.25   ]

# Ve = [ 12014.375  , 11630.9375 , 13548.125   ,15081.875  , 14826.25  ,  16232.1875  ]

# ,10007 1990-01-,31 00:00:00 
r = 8.08
X = 396.981 



In [57]:
prova = calc_DD(r, X, Ve, 1)
prova
# sigma_a=calcVolatility(Ve)
# sigma_a

C:\Users\Huang\AppData\Local\Temp\ipykernel_25448\3866772091.py:11: RuntimeWarning: divide by zero encountered in divide
  return np.std(np.log(np.divide(x[1:],x[:-1])))*np.sqrt(252)
C:\Users\Huang\AppData\Local\Temp\ipykernel_25448\3866772091.py:11: RuntimeWarning: divide by zero encountered in log
  return np.std(np.log(np.divide(x[1:],x[:-1])))*np.sqrt(252)
c:\Users\Huang\OneDrive - Università Commerciale Luigi Bocconi\python\macro-fundamental-factors\.env\Lib\site-packages\numpy\core\_methods.py:215: RuntimeWarning: invalid value encountered in reduce
  arrmean = umr_sum(arr, axis, dtype, keepdims=True, where=where)


UnboundLocalError: cannot access local variable 'xguess' where it is not associated with a value

In [62]:
asdf = pd.read_csv("..\..\data\pulled\wrds_crsp_d.csv", parse_dates=['date'])
# asdf['me'] = asdf['me'].replace(0, np.nan)
# asdf = asdf.dropna(subset=['me'], how='any')